In [3]:
import mne
import json
from pathlib import Path
import sys

'''Gathering all epochs in a combined one'''
# Define paths and read session info as in your provided code...

# Assuming notebook is located in: /work/name/notebooks_PHB/MEG_portfolio/sanity_checks/
notebook_path = Path("/work/PernilleHøjlundBrams#8577/notebooks_PHB/MEG_portfolio/sanity_checks")

# Add the utils module path to sys.path to import preprocess_data_sensorspace
sys.path.append(str(notebook_path.parents[0]))  # Adding the parent directory to sys.path
from utils import preprocess_data_sensorspace

def gather_all_epochs(subjects, recording_names, session_info, MEG_data_path, ICA_path):
    all_epochs = []

    for subject in subjects:
        subject_info = session_info[subject]
        reject = subject_info["reject"]

        subject_path = MEG_data_path / subject
        subject_meg_path = list(subject_path.glob("*_000000"))[0]

        # Looping through blocks
        for recording_name in recording_names:

            subject_session_info = subject_info[recording_name]
            fif_file_path = list((subject_meg_path / "MEG" / recording_name / "files").glob("*.fif"))[0]
            ICA_path_sub = ICA_path / subject / f"{recording_name}-ica.fif"

            # If a self-block, then take only 11/12 (no buttons)
            if 'self' in recording_name:
                event_id = {"img/self/positive": 11, "img/self/negative": 12}
            # If an other-block, then take only 21/22 (no buttons)
            elif 'other' in recording_name: 
                event_id = {"img/assigned/positive": 21, "img/assigned/negative": 22}

            epochs = preprocess_data_sensorspace(
                fif_file_path,
                subject_session_info["bad_channels"],
                reject,
                ICA_path_sub,
                subject_session_info["noise_components"],
                event_ids = event_id
            )
            all_epochs.append(epochs)

    # Concatenate all epochs into a single Epochs object
    combined_epochs = mne.concatenate_epochs(all_epochs, add_offset=True)
    
    return combined_epochs


In [6]:

# Check P1
import mne
import matplotlib.pyplot as plt
import numpy as np

# Read session info, defining subjs and recordings, paths
with open('/work/PernilleHøjlundBrams#8577/notebooks_PHB/MEG_portfolio/session_info.txt', 'r') as f:
    session_info = json.loads(f.read())

# Just trying one now
#subjects = ["0108", "0109", "0110", "0111", "0112", "0113", "0114", "0115"]
subjects = ["0108"]
recording_names = [
    '001.self_block1',  '002.other_block1', '003.self_block2',
    '004.other_block2', '005.self_block3',  '006.other_block3'
]

# Define other paths
MEG_data_path = Path("/work/834761")
ICA_path = Path("/work/study_group_8/ICA")  # Adjusting according to yo

# Gather all epochs
combined_epochs = gather_all_epochs(subjects, recording_names, session_info, MEG_data_path, ICA_path)

# For magnetometer data, you might want to select relevant channels (e.g., those over occipital areas)
# You can modify this selection based on your experimental setup and sensors' layout
occipital_chs = mne.pick_channels_regexp(epochs.ch_names, regexp='M[LOZ12]*-?3')

# Epochs selection if some rejection is needed
# epochs = epochs.drop_bad() # if you've set rejection parameters but not applied them yet
# epochs = epochs['your_condition'] # if you want to select a specific condition

# Extract an ERP/ERF for a specific condition and channels
# Here, we average across epochs (assuming event_id=1, adjust as needed), 
# and then select the channels of interest.
erf = epochs.average().pick_channels([epochs.ch_names[i] for i in occipital_chs])

# Plot the ERF with the time range where you expect the P1 (e.g., 80 to 120 ms)
erf.plot(titles='Occipital Channels', time_unit='s', 
         spatial_colors=True, picks='mag',
         show=False)
plt.axvline(x=0.1, color='k', linestyle='--', label='100 ms')
plt.axvline(x=0.12, color='r', linestyle='--', label='120 ms')
plt.legend()
plt.show()

# Get the Global Field Power (GFP) and plot it
gfp = np.sqrt((erf.data ** 2).mean(axis=0))
plt.plot(erf.times, gfp, label='GFP')
plt.axvline(x=0.1, color='k', linestyle='--', label='100 ms')
plt.axvline(x=0.12, color='r', linestyle='--', label='120 ms')
plt.title('Global Field Power (GFP)')
plt.xlabel('Time (s)')
plt.ylabel('GFP')
plt.legend()
plt.show()

# For statistical analysis of the P1 across trials, you might look at a specific 
# time window and channel and perform a t-test to see if the activity is 
# significantly different from baseline or zero.

# Example: check if the activity between 100-120 ms is different from baseline
from scipy.stats import ttest_1samp

# Extract data in the 100-140 ms window
time_mask = np.logical_and(erf.times >= 0.1, erf.times <= 0.14)
data_in_window = epochs.get_data(picks=occipital_chs)[:, :, time_mask].mean(axis=-1)

# Statistical test
t_stat, p_val = ttest_1samp(data_in_window, popmean=0, axis=0)
print(f"t-stat: {t_stat}, p-value: {p_val}")

# Note: Ensure the analysis & tests fit your experimental design & hypothesis, 
# this is a simple general check for the P1.




Opening raw data file /work/834761/0108/20230928_000000/MEG/001.self_block1/files/self_block1.fif...
    Read a total of 9 projection items:
        planar-Raw-0.000-120.000-PCA-01 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-02 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-03 (1 x 203)  idle
        axial-Raw-0.000-120.000-PCA-01 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-02 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-03 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-04 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-05 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-06 (1 x 102)  idle
    Range : 74000 ... 452999 =     74.000 ...   452.999 secs
Ready.
Reading 0 ... 378999  =      0.000 ...   378.999 secs...


/work/PernilleHøjlundBrams#8577/notebooks_PHB/MEG_portfolio/utils.py:29: RuntimeWarning: This filename (/work/834761/0108/20230928_000000/MEG/001.self_block1/files/self_block1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 306 out of 306 | elapsed:    3.4s finished


Reading /work/study_group_8/ICA/0108/001.self_block1-ica.fif ...
    Read a total of 9 projection items:
        planar-Raw-0.000-120.000-PCA-01 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-02 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-03 (1 x 203)  idle
        axial-Raw-0.000-120.000-PCA-01 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-02 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-03 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-04 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-05 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-06 (1 x 102)  idle
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance
    Transforming to ICA space (203 components)
    Zeroing out 3 ICA components
    Projecting back using 305 PCA components
158 events found
Event IDs: [ 11  12  23 202]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (sub

...

Reading /work/study_group_8/ICA/0108/006.other_block3-ica.fif ...


[Parallel(n_jobs=4)]: Done 306 out of 306 | elapsed:    3.6s finished


    Read a total of 9 projection items:
        planar-Raw-0.000-120.000-PCA-01 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-02 (1 x 203)  idle
        planar-Raw-0.000-120.000-PCA-03 (1 x 203)  idle
        axial-Raw-0.000-120.000-PCA-01 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-02 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-03 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-04 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-05 (1 x 102)  idle
        axial-Raw-0.000-120.000-PCA-06 (1 x 102)  idle
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance
    Transforming to ICA space (183 components)
    Zeroing out 3 ICA components
    Projecting back using 304 PCA components
160 events found
Event IDs: [ 21  22  23 202]
Not setting metadata
80 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 9)
9 projection items activated
Using data from

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 22794 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 25680 out of 25680 | elapsed:    1.6s finished


ValueError: epochs[1].info['dev_head_t'] differs. The instances probably come from different runs, and are therefore associated with different head positions. Manually change info['dev_head_t'] to avoid this message but beware that this means the MEG sensors will not be properly spatially aligned. See mne.preprocessing.maxwell_filter to realign the runs to a common head position.